In [1]:
# first scrape stations

import requests
import pandas as pd

# Define the base URL for fetching data
base_url = "https://data.smartdublin.ie/sonitus-api/api/monitors?username=dublincityapi&password=Xpa5vAQ9ki"

try:
    # Make the request to fetch noise averages data
    response = requests.post(base_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (status codes >= 400)

    # Extract data from the response
    data = response.json()

    # Create lists to store data
    serial_numbers = []
    labels = []
    locations = []
    latitudes = []
    longitudes = []
    last_calibrated_dates = []

    # Extracting data from the response
    for station in data:
        serial_numbers.append(station["serial_number"])
        labels.append(station["label"])
        locations.append(station["location"])
        latitudes.append(station["latitude"])
        longitudes.append(station["longitude"])
        last_calibrated_dates.append(station["last_calibrated"])

    # Create DataFrame
    station_data = pd.DataFrame({
        "serial_number": serial_numbers,
        "label": labels,
        "location": locations,
        "latitude": latitudes,
        "longitude": longitudes,
        "last_calibrated": last_calibrated_dates
    })

    #print(df.head())  # Display the first few rows of the DataFrame

except requests.RequestException as e:
    print("Error fetching data:", e)
except Exception as e:
    print("An unexpected error occurred:", e)

In [3]:
# Filter the DataFrame to only include air pollution stations
air_pollution_stations = station_data[station_data["label"].str.contains("Air")]

In [7]:
# count unique stations
unique_stations = air_pollution_stations["serial_number"].nunique()
print(f"Number of unique air pollution stations: {unique_stations}")

Number of unique air pollution stations: 34


Approach 2: Split up time period into smaller chunks

In [40]:
import datetime

# Define the base URL for fetching noise averages data
base_url_hourly_averages = 'https://data.smartdublin.ie/sonitus-api/api/hourly-averages'

def split_time_period(start_date, end_date, delta_days=7):
    current_date = start_date
    while current_date < end_date:
        next_date = min(end_date, current_date + datetime.timedelta(days=delta_days))
        yield (int(current_date.timestamp()), int(next_date.timestamp()))
        current_date = next_date  # This ensures the loop advances to the next period.

# Ensure you are using the function within the correct date range:
start_date = datetime.datetime.strptime('2021-05-06 07:35:36', '%Y-%m-%d %H:%M:%S')
end_date = datetime.datetime.strptime('2022-08-05 15:27:42', '%Y-%m-%d %H:%M:%S')

# Example use of the corrected function:
serial_number = "DCC-AQ1"
for start_ts, end_ts in split_time_period(start_date, end_date):
    url = f"{base_url_hourly_averages}?username=dublincityapi&password=Xpa5vAQ9ki&monitor={serial_number}&start={start_ts}&end={end_ts}"
    response = requests.post(url)
    if response.status_code == 200:
        data = response.json()
        print(data)
        # Process and store data

[{'datetime': '2021-05-06 06:00:00', 'co': 0.36, 'no': 23.54, 'no2': 46.08, 'so2': 2.76}, {'datetime': '2021-05-06 07:00:00', 'co': 0.36, 'no': 13.75, 'no2': 31.34, 'so2': 2.17}, {'datetime': '2021-05-06 08:00:00', 'co': 0.36, 'no': 9.91, 'no2': 22.33, 'so2': 2.23}, {'datetime': '2021-05-06 09:00:00', 'co': 0.39, 'no': 5.17, 'no2': 10.81, 'so2': 2.67}, {'datetime': '2021-05-06 10:00:00', 'co': 0.44, 'no': 4.96, 'no2': 9.46, 'so2': 2.47}, {'datetime': '2021-05-06 11:00:00', 'co': 0.35, 'no': 4.52, 'no2': 8.98, 'so2': 2.03}, {'datetime': '2021-05-06 12:00:00', 'co': 0.35, 'no': 3.92, 'no2': 8.17, 'so2': 2.08}, {'datetime': '2021-05-06 13:00:00', 'co': 0.35, 'no': 5.33, 'no2': 10.65, 'so2': 2.07}, {'datetime': '2021-05-06 14:00:00', 'co': 0.36, 'no': 7.12, 'no2': 14.14, 'so2': 2.22}, {'datetime': '2021-05-06 15:00:00', 'co': 0.36, 'no': 5.29, 'no2': 13.71, 'so2': 1.82}, {'datetime': '2021-05-06 16:00:00', 'co': 0.36, 'no': 5.61, 'no2': 12.53, 'so2': 1.73}, {'datetime': '2021-05-06 17:00:0

KeyboardInterrupt: 

In [4]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

# Setup API parameters and directories
url = 'https://data.smartdublin.ie/sonitus-api/api/hourly-averages'
headers = {'accept': 'application/json'}
username = 'dublincityapi'
password = 'Xpa5vAQ9ki'
base_directory = "Air_Pollution_Data"

# Assuming 'air_pollution_stations' is your DataFrame with all stations and their metadata
# Ensure that 'serial_number' is a column in your DataFrame
serial_numbers = air_pollution_stations['serial_number'].unique()
print(serial_numbers)

['DCC-AQ1' 'DCC-AQ2' 'DCC-AQ3' 'DCC-AQ4' 'DCC-AQ5' 'DCC-AQ6' 'DCC-AQ7'
 'DCC-AQ8' 'DCC-AQ9' 'DCC-AQ10' 'TNT1088' 'TNT1138' 'TNT1296' 'TNO2161'
 'TNO2162' 'DCC-AQ13' 'DCC-AQ17' 'DCC-AQ22' 'DCC-AQ52' 'DCC-AQ69'
 'TNO4435' 'TNO4438' 'TNO4488' 'TNO4390' 'TNO4324' 'TNO4323' 'TNO4325'
 'TNO4437' '0110-000157-000000' '0110-000180-000000' '0110-000141-000000'
 'DM30-00530' 'DM30-00531' 'DCC-AQ91']


In [5]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta
import json

# Setup API parameters and directories
url = 'https://data.smartdublin.ie/sonitus-api/api/hourly-averages'
headers = {'accept': 'application/json'}
username = 'dublincityapi'
password = 'Xpa5vAQ9ki'

# Assuming 'air_pollution_stations' is your DataFrame with all stations and their metadata
serial_numbers = air_pollution_stations['serial_number'].unique()
start_date = datetime(2021, 5, 1)
end_date = datetime(2022, 8, 31)

# Function to generate date ranges
def generate_date_ranges(start_date, end_date, delta_days=15):
    current_date = start_date
    while current_date < end_date:
        next_date = min(end_date, current_date + timedelta(days=delta_days))
        yield current_date, next_date
        current_date = next_date


# Collect all dataframes here
all_data = []

for serial in serial_numbers:
    for start, end in generate_date_ranges(start_date, end_date):
        params = {
            'username': username, 'password': password,
            'monitor': serial, 'start': int(start.timestamp()), 'end': int(end.timestamp())
        }
        response = requests.post(url, headers=headers, params=params)
        if response.status_code == 200 and response.text:
            try:
                data = response.json()
                if data:  # Ensure data is not empty
                    df = pd.DataFrame(data)
                    df['serial_number'] = serial  # Add serial number for merging
                    all_data.append(df)
                else:
                    print(f"No data for {serial} from {start} to {end}")
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON response for {serial} from {start} to {end}")
        else:
            print(f"Request failed for {serial} from {start} to {end}, Status Code: {response.status_code}")

# Combine and join with station data
combined_df = pd.concat(all_data, ignore_index=True, sort=False)
final_df = pd.merge(air_pollution_stations, combined_df, on="serial_number", how='left')

print("Data retrieval complete and saved successfully.")

No data for DCC-AQ3 from 2021-09-13 00:00:00 to 2021-09-28 00:00:00
Request failed for DCC-AQ4 from 2022-02-10 00:00:00 to 2022-02-25 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-02-25 00:00:00 to 2022-03-12 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-03-12 00:00:00 to 2022-03-27 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-03-27 00:00:00 to 2022-04-11 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-04-11 00:00:00 to 2022-04-26 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-04-26 00:00:00 to 2022-05-11 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-05-11 00:00:00 to 2022-05-26 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-05-26 00:00:00 to 2022-06-10 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-06-10 00:00:00 to 2022-06-25 00:00:00, Status Code: 429
Request failed for DCC-AQ4 from 2022-06-25 00:00:00 to 2022-07-10 00:00:00, Status Code: 429
Re

In [6]:
# drop columns 'last_calibrated' and 'location' and 'label'
df = final_df.copy()
df.drop(columns=['last_calibrated', 'location', 'label'], inplace=True)

# Convert 'datetime' to datetime type and set as index
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Define the value columns of interest
value_columns = ['pm1', 'pm10', 'pm2_5', 'pm4', 'tsp', 'o3', 'co', 'no', 'no2', 'so2']

# Use pivot_table to reshape the DataFrame
df_pivot = df.pivot_table(index=df.index, columns='serial_number', values=value_columns, aggfunc='first')

# Flatten the columns by combining serial number with value column names
df_pivot.columns = [f'{col[1]}-{col[0]}' for col in df_pivot.columns]

# Drop the original measurement columns and the 'serial_number'
df_pivot.reset_index(inplace=True)  # if you need the datetime index reset into a column

# Print or return the transformed DataFrame
df_pivot

,datetime,DCC-AQ1-co,DCC-AQ1-no,DCC-AQ10-no,DCC-AQ13-no,DCC-AQ5-no,DCC-AQ6-no,DCC-AQ1-no2,DCC-AQ10-no2,DCC-AQ13-no2,...,TNO4323-tsp,TNO4324-tsp,TNO4325-tsp,TNO4390-tsp,TNO4435-tsp,TNO4437-tsp,TNO4438-tsp,TNO4488-tsp,TNT1088-tsp,TNT1296-tsp
0,2021-04-30 22:00:00,0.47,6.54,43.11,27.77,0.46,13.82,68.00,63.11,-1.47,...,NaN,NaN,12.85,NaN,16.67,NaN,13.75,14.42,5.73,NaN
1,2021-04-30 23:00:00,0.55,8.21,26.41,46.69,8.96,8.39,60.76,58.34,2.92,...,NaN,NaN,14.57,NaN,19.75,NaN,39.72,17.35,7.63,NaN
2,2021-05-01 00:00:00,0.49,2.19,18.79,43.65,0.76,3.42,52.77,60.68,3.44,...,NaN,NaN,14.05,NaN,17.40,NaN,18.57,14.80,9.20,NaN
3,2021-05-01 01:00:00,0.47,5.92,13.31,34.25,2.09,1.65,51.68,50.40,-0.89,...,NaN,NaN,15.40,NaN,10.92,NaN,17.20,11.80,4.78,NaN
4,2021-05-01 02:00:00,0.39,2.32,6.11,35.84,0.68,0.80,42.85,32.74,2.60,...,NaN,NaN,13.35,NaN,12.33,NaN,12.35,6.73,3.75,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11681,2022-08-30 18:00:00,NaN,-1.89,10.80,NaN,0.27,1.38,4.40,22.11,NaN,...,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11682,2022-08-30 19:00:00,NaN,-1.35,9.86,NaN,0.17,-1.24,5.75,17.92,NaN,...,8.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11683,2022-08-30 20:00:00,NaN,-2.32,17.94,NaN,-0.38,-3.10,2.16,24.20,NaN,...,10.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11684,2022-08-30 21:00:00,NaN,-2.18,7.86,NaN,-0.47,-2.72,2.13,16.95,NaN,...,9.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# export to csv to 'dublin_official_measurements_sonitus_api.csv'
df_pivot.to_csv('dublin_official_measurements_sonitus_api.csv', index=False)

In [12]:
# get stats of missingness per column
# find columns where missing data is >  20%
missing_data = df_pivot.isnull().mean() * 100
missing_data[missing_data > 20]

DCC-AQ17-no2                65.377375
DCC-AQ69-no2                32.449084
DCC-AQ69-o3                 33.552969
DCC-AQ17-pm1                65.223344
DCC-AQ4-pm1                 40.450111
DCC-AQ69-pm1                60.320041
TNO4323-pm1                 49.640596
TNO4324-pm1                 66.267328
TNO4325-pm1                 60.088995
TNO4390-pm1                 64.572993
TNO4435-pm1                 38.353585
TNO4437-pm1                 67.918877
TNO4438-pm1                 81.644703
TNO4488-pm1                 66.019168
TNT1296-pm1                 73.404073
0110-000141-000000-pm10     87.617662
0110-000157-000000-pm10     83.784015
0110-000180-000000-pm10     84.776656
DCC-AQ17-pm10               65.223344
DCC-AQ22-pm10               24.910149
DCC-AQ4-pm10                40.450111
DCC-AQ69-pm10               60.320041
TNO4323-pm10                55.356837
TNO4324-pm10                69.056991
TNO4325-pm10                60.088995
TNO4390-pm10                64.572993
TNO4435-pm10